# Configuring igpu for xserver and Nvidia gpu for CUDA Ubuntu 18.04 
----

### This step by step guide will help you install Nvidia GPU to work on CUDA only, and for the integrated gpu to take care of the displays.
Bionic Beaver (Ubuntu 18.04) is shipped with both the traditional Xorg graphics stack as well as the newer Wayland based stack, but Xorg will be the default. This [article](https://blog.ubuntu.com/2018/01/26/bionic-beaver-18-04-lts-to-use-xorg-by-default) written by Will Cooke a Desktop Engineering Manager explains why Xorg is not the default on Bionic Beaver, in contrast with the previous Ubuntu 17.04 version.

In general Xorg is the X Window server which allows users to have a graphical environment at their fingertips. A thorough guide that explains what Xorg is can be found [here](https://wiki.gentoo.org/wiki/Xorg/Guide). [Wayland](https://wayland.freedesktop.org/) is intended as a simpler replacement for X, easier to develop and maintain.


To check whether your Ubuntu is running on Wayland or Xorg, open a terminal use:

        > cat /etc/gdm3/custom.conf
To disable Wayland access the configuration file with an editor of your choice using **sudo** and **un-comment** the line, and reboot the system to apply changes.
    
        > #WaylandEnable=false


To check what hardware is in our system:

    > sudo lshw -c display

![Display](images/displays.png)

From the image above we can see that there are 2 hardware devices related to the display.

What is important for us is to see that the driver installed for the GPU is the [nouveau](https://nouveau.freedesktop.org/wiki/) driver which is a free and open-source graphics device driver for nvidia video cards. You can also check which type of driver you are using from the software and update center under additional drivers.

![Drivers](images/SU_ss.png)



----
## 1 - Installing Via Nvidia run files (recommended)

Make sure that the BIOS is configured to drive the monitor from iGPU.

[Download](https://www.nvidia.com/Download/index.aspx?lang=en-us) the driver version directly from the nvidia website by inputting your details and run the file.

    > sudo ./NVIDIA-Linux-x86_64-390.87.run --no-opengl-files --no-x-check --disable-nouveau
    
where:
    
-  --no-opengl-files is the key option that makes this work. We don't want to overwrite the Intel opengl setup which is important for compiz.
-  --no-x-check allows us to install the driver without stopping X.
-  --disable-nouveau is there just in case you do want to drive the display from a gpu at some point.

When prompted don't configure X.

If you're under UEFI have the installer generate a key pair for you and save those for future driver updates. Put the pub key on a usb stick and append it to the DB store in your bios.

To identify if the computer boots the HDD in UEFI mode:

    > [ -d /sys/firmware/efi ] && echo "EFI boot on HDD" || echo "Legacy boot on HDD" 
    
Now we will use the [Persistence Daemon](https://docs.nvidia.com/deploy/driver-persistence/index.html#persistence-daemon-supported-environments) to keep the driver up and running. Clients attach a GPU by opening its device file and GPU is detached by closing the device file. If all clients have closed the device file, the GPU state will be unloaded unless persistence mode is enabled. 

To run the NVIDIA Persistence Daemon, run as root:

    > nvidia-persistenced --user foo
    
To install the NVIDIA Persistence Daemon to start on system initialization please check [here](https://docs.nvidia.com/deploy/driver-persistence/index.html#installation) .
    
** Potential problems with ubuntu updates.


## 2 - Installing Via GUI

On Ubuntu 18.04, setting up the nvidia driver is fairly simple. It can be done through the graphical user interface above. You can compare the driver number with the one listed on the [nvidia website](https://www.nvidia.com/Download/index.aspx) by choosing your system details. It is recommended to **disable** secure boot if you are using UEFI firmware, some problems might be encountered if it is enabled.



## 3 - Installing Via Command line

First by adding the Nvidia PPA:

    > sudo add-apt-repository ppa:graphics-drivers/ppa
Then:

    > sudo apt install nvidia-driver-**driver-version**
    
In our case:
    
    > sudo apt install nvidia-driver-390
    
----

# Installing CUDA on Ubuntu 18.04

To check which cuda version is compatible with the driver you just installed please check this [post](https://stackoverflow.com/questions/30820513/what-is-version-of-cuda-for-nvidia-304-125/30820690#30820690). 

In our case for a 390 driver version, CUDA 9.1 is the compatible one.

The CUDA development environment relies on tight integration with the host development environment, including the host compiler and C runtime libraries and is therefore only supported on distribution versions that have been qualified for this CUDA Toolkit release. Full documentation can be found [here](https://docs.nvidia.com/cuda/archive/9.1/cuda-installation-guide-linux/index.html).

![CUDA](images/cuda_dependencies.png)

Since the default gcc version of ubuntu 18.04 is 7.3.0, it needs to be changed. To check what version of gcc you are using:

    > gcc --version
    
![gcc](images/gcc.png)

The default kernel version of ubuntu 18.04 is 4.15. To check the version use:
    
    > uname -r

![kernel](images/kernel.png)

To check the ldd version:
    
    > ldd --version

![ldd](images/ldd.png)

A solution for the gcc/g++ compatibility is to remove the gcc/g++7.3 and install the correct version required, but a yet better solution is to use the update-alternatives that allows you to switch between versions.

Remove current alternatives:


    > sudo update-alternatives --remove-all gcc 
    > sudo update-alternatives --remove-all g++
    
Install packages:

    > sudo apt-get install gcc-6 g++-6
    
Install Alternatives:
       
    > update-alternatives: --install needs (link) (name) (path) (priority)

       
    > sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-6 10
    > sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 20
    
    > sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-6 10
    > sudo update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-7 20
    
    > sudo update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
    > sudo update-alternatives --set cc /usr/bin/gcc
    > sudo update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
    > sudo update-alternatives --set c++ /usr/bin/g++
    
Configure Alternatives:

    > sudo update-alternatives --config gcc
    > sudo update-alternatives --config g++
    
Installing other CUDA dependencies:

    > sudo apt-get install freeglut3 freeglut3-dev libxi-dev libxmu-dev

## CUDA run file installer

Since Nvidia did not include an installer for Ubuntu 18.04, we are going to use the run files of CUDA toolkit 9.1 for ubuntu 17.04.

Head to the [CUDA toolkit archive](https://developer.nvidia.com/cuda-91-download-archive) and download the runfile installer and all the patches.

![cuda_runfile](images/cuda_runfile.png)

Make sure that driver version on the CUDA runfile name is the one supported (It is not necessary for them to be identical). For example here it is 387.26 but our driver version is 390.xx. You can then run the runfile you just downloaded using:

    > sudo sh cuda_9.1.85_387.26_linux.run
    
    
You will be prompted with several questions one of which is:


Install NVIDIA Accelerated Graphics Driver for Linux-x86_64 387.26? y/n/q


    > Make sure to answer **n**, as we already installed the nvidia driver
    
  
Enter CUDA Samples Location:

        > You can set it to **/usr/local/cuda-9.1** if you don't want it in your home directory

Install the patches by order:

    > sudo sh cuda_9.1.85.1_linux.run
    > sudo sh cuda_9.1.85.2_linux.run
    > sudo sh cuda_9.1.85.3_linux.run
    


## Setting up environment variables (System wide)

Create the file cuda.sh:

    > sudo touch /etc/profile.d/cuda.sh

Add the following commands inside the cuda.sh file:

    > echo $'export PATH=$PATH:/usr/local/cuda/bin \nexport CUDADIR=/usr/local/cuda' | sudo tee --append /etc/profile.d/cuda.sh
    
Create the following file as well:

    > sudo touch /etc/ld.so.conf.d/cuda.conf
    
Add the following command:

    > echo '/usr/local/cuda/lib64' | sudo tee --append /etc/ld.so.conf.d/cuda.conf
    
Run:

    > sudo ldconfig
    
And finally:

    > source /etc/profile.d/cuda.sh

----
## CUDNN

Install [cuDNN](https://developer.nvidia.com/cudnn) version that is compatible with the version of CUDA you just downloaded and as well as with TensorFlow.

![TFcomp](images/tfcompt.png)

Since we will be installing tensorflow v1.10 later, we will go with cuDNN 7. Looking into the cuDNN archive, download cuDNN v7.1.3 (April 17, 2018), for CUDA 9.1 .
Extract the .tgz file any where you like, let's assume it is in your home directory for simplicity. Change **direc** to your own direc **name**.

    > sudo cp /home/direc/cudnn-9.1-linux-x64-v7.1/cuda/include/* /usr/local/cuda-9.1/include/
    > sudo cp /home/direc/cudnn-9.1-linux-x64-v7.1/cuda/lib64/libcudnn.so.7.1.3 /home/direc/cudnn-9.1-linux-x64-v7.1/cuda/lib64/libcudnn_static.a /usr/local/cuda-9.1/lib64/
    > sudo chmod a+r /usr/local/cuda-9.1/include/cudnn.h /usr/local/cuda-9.1/lib64/libcudnn*
    > cd /usr/lib/x86_64-linux-gnu
    > sudo ln -s libcudnn.so.7.1.4 libcudnn.so.7
    > sudo ln -s libcudnn.so.7 libcudnn.so
    

----
## Installing NCCL

Head to the [NCCL](https://developer.nvidia.com/nccl/nccl-download) website and download the tar file corresponding to the installations above. We will choose from Download NCCL v2.1.15, for **CUDA 9.1, March 8, 2018,** and then click on **NCCL 2.1.15 O/S agnostic and CUDA 9**.

Extract the .txz file any where you like, let's assume it is in your home directory for simplicity. Change direc to your own direc name.

    > sudo mkdir -p /usr/local/cuda-9.1/nccl/lib /usr/local/cuda-9.1/nccl/include
    > sudo cp /home/direc/nccl_2.1.15-1+cuda9.1_x86_64/*.txt /usr/local/cuda-9.1/nccl
    > sudo cp /home/direc/nccl_2.1.15-1+cuda9.1_x86_64/include/*.h /usr/include/
    > sudo cp /home/direc/nccl_2.1.15-1+cuda9.1_x86_64/lib/libnccl.so.2.1.15 /home/direc/nccl_2.1.15-1+cuda9.1_x86_64/lib/libnccl_static.a /usr/lib/x86_64-linux-gnu/
    > sudo ln -s /usr/include/nccl.h /usr/local/cuda-9.1/nccl/include/nccl.h
    > cd /usr/lib/x86_64-linux-gnu
    > sudo ln -s libnccl.so.2.1.15 libnccl.so.2
    > sudo ln -s libnccl.so.2 libnccl.so
    > for i in libnccl*; do sudo ln -s /usr/lib/x86_64-linux-gnu/$i /usr/local/cuda-9.1/nccl/lib/$i; done


----
## Installing Bazel

Installing Bazel using [Bazel custom APT repo](https://docs.bazel.build/versions/master/install-ubuntu.html#install-on-ubuntu):

    > sudo apt-get install openjdk-8-jdk
    > echo "deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
    > curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
    > sudo apt-get update && sudo apt-get install bazel
    
Upgrading to a new version

    > sudo apt-get upgrade bazel

----
## Installing Tensorflow

We will set up tensorflow using the repository directly:

    > git clone https://github.com/tensorflow/tensorflow
    > cd tensorflow
    > ./configure
    
You will be prompted by a number of inputs that you will have to specify depending on what you need:

The most important ones are:
    
    > Do you wish to build TensorFlow with CUDA support? [y/N]: y
    > Please specify the CUDA SDK version you want to use, e.g. 7.0. [Leave empty to default to CUDA 9.0]: 9.1
    > Please specify the location where CUDA 9.1 toolkit is installed. Refer to README.md for more details. [Default is /usr/local/cuda]: /usr/loca/cuda-9.1
    > Please specify the cuDNN version you want to use. [Leave empty to default to cuDNN 7.0]: 7.1
    > Please specify the location where cuDNN 7 library is installed. Refer to README.md for more details. [Default is /usr/local/cuda]: /usr/local/cuda-9.1
    > Please specify the NCCL version you want to use. [Leave empty to default to NCCL 1.3]: 2.1.15
    > Please specify the location where NCCL 2 library is installed. Refer to README.md for more details. [Default is /usr/local/cuda]:/usr/local/cuda-9.1/nccl
    > Please specify a list of comma-separated Cuda compute capabilities you want to build with. You can find the compute capability of your device at: https://developer.nvidia.com/cuda-gpus. Please note that each additional compute capability significantly increases your build time and binary size. [Default is: 3.5,5.2]: 6.1 (The one belonging to your gpu)
    > Please specify which gcc should be used by nvcc as the host compiler. [Default is /usr/bin/x86_64-linux-gnu-gcc-7]: /usr/bin/gcc-6 (Could be 4.8, it is the one recommended on TF website)
    
 ----   
 
    > pip install keras
    > bazel build --config=opt --config=cuda //tensorflow/tools/pip_package:build_pip_package
    > bazel-bin/tensorflow/tools/pip_package/build_pip_package /tmp/tensorflow_pkg
    > pip install /tmp/tensorflow_pkg/tensorflow*.whl

----
## Credits:

GCC alternatives:

https://askubuntu.com/questions/26498/how-to-choose-the-default-gcc-and-g-version

CUDA installation: 

https://www.pugetsystems.com/labs/hpc/How-to-install-CUDA-9-2-on-Ubuntu-18-04-1184/#step-3-install-cuda-dependencies

Manual Nvidia installation:

https://devtalk.nvidia.com/default/topic/964757/setting-up-a-linux-gpu-dev-box-with-integrated-graphics-driving-the-display/

Ubuntu UEFI:

https://help.ubuntu.com/community/UEFI

cuDNN:

https://medium.com/@asmello/how-to-install-tensorflow-cuda-9-1-into-ubuntu-18-04-b645e769f01d

Bazel:

https://docs.bazel.build/versions/master/install-ubuntu.html#install-on-ubuntu